In [6]:
import json,os
from PIL import Image, ImageDraw, ImageFont

In [7]:
def read_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            # 解析每一行的 JSON 对象
            json_obj = json.loads(line.strip())
            data.append(json_obj)
    return data

def read_json(file_path):
    with open(file_path, 'r') as f:
        rslt = json.load(f)
    return rslt

In [9]:
rslt_path = "../experiments/changechat-lora-gpt-114k/results/ans_20240810_121543.jsonl"
gt_path = "/root/autodl-tmp/LEVIR-MCI-dataset/LevirCCcaptions-v2.json"
results = read_jsonl(rslt_path)
gts = read_json(gt_path)
gts_dict = {item['filename']: item for item in gts}

In [10]:
def combine_images_with_text(image1_path, image2_path, text_lines, output_path):
    # 打开图像
    image1 = Image.open(image1_path)
    image2 = Image.open(image2_path)
    
    # 获取图像尺寸
    width1, height1 = image1.size
    width2, height2 = image2.size
    
    # 设置文本字体和行高
    font = ImageFont.load_default()
    line_height = font.getsize('A')[1] + 2  # 每行文本的高度，包括一些间距
    text_height = line_height * len(text_lines)  # 文本的总高度
    
    # 创建一个新的空白图像，宽度为两幅图像的宽度之和，高度为两幅图像的最大高度加上文本的高度
    total_width = width1 + width2
    max_height = max(height1, height2)
    combined_image = Image.new('RGB', (total_width, max_height + text_height), (255, 255, 255))
    
    # 将第一幅图像粘贴到新图像上
    combined_image.paste(image1, (0, text_height))
    # 将第二幅图像粘贴到新图像上
    combined_image.paste(image2, (width1, text_height))
    
    # 在图像上绘制文本
    draw = ImageDraw.Draw(combined_image)
    text_x = 10  # 文本距离左边的距离
    for i, line in enumerate(text_lines):
        text_y = i * line_height  # 每行文本的 y 坐标
        draw.text((text_x, text_y), line, fill="black", font=font)
    
    # 保存最终图像
    combined_image.save(output_path)
    print(f"Combined image saved to {output_path}")


In [11]:
rslt_path = '/root/autodl-tmp/LEVIR-MCI-dataset/results-0810'
rslt_path_no_c = rslt_path + '/no_change'
rslt_path_c = rslt_path + '/change'

for row in results:
    image1_path = os.path.join("/root/autodl-tmp/LEVIR-MCI-dataset/images", row['image_id'][0])
    image2_path = os.path.join("/root/autodl-tmp/LEVIR-MCI-dataset/images", row['image_id'][1])
    filename = row['image_id'][0].split('/')[-1]
    if row['answer'].replace('.','').strip().lower() in [x.replace('.','').strip() for x in gts_dict.get(filename)['sentences']]:
        continue
    textLines = []
    textLines.append("Labels:")
    textLines.extend([' - ' + x for x in gts_dict.get(filename)['sentences']])
    textLines.append('OURS:' + row['answer'])
    
    if gts_dict.get(filename)['changeflag'] == 0:
        _r_path  = rslt_path_no_c
    else:
        _r_path = rslt_path_c
    if not os.path.exists(_r_path):
        os.makedirs(_r_path)
    combine_images_with_text(image1_path, image2_path, textLines, os.path.join(_r_path, filename))


/tmp/ipykernel_9360/518700781.py:12: DeprecationWarning: getsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use getbbox or getlength instead.
  line_height = font.getsize('A')[1] + 2  # 每行文本的高度，包括一些间距


Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000004.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000017.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000018.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000019.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000020.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000031.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000032.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/no_change/test_000033.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/no_change/test_000041.png
Combined image saved to /root/autodl-tmp/LEVIR-MCI-dataset/results-0810/change/test_000044.png
Combined image saved to /root/autodl-tmp/LEV